In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import cv2
import os
import pandas as pd
import h5py


train_folder = "train"
test_folder = "test"
extra_folder = "<YOURDIR>\extra"
resize_size = (64,64)


def get_name(index, hdf5_data):
    name = hdf5_data['/digitStruct/name']
    return ''.join([chr(v[0]) for v in hdf5_data[name[index][0]].value])

def get_bbox(index, hdf5_data):
    attrs = {}
    item = hdf5_data['digitStruct']['bbox'][index].item()
    for key in ['label', 'left', 'top', 'width', 'height']:
        attr = hdf5_data[item][key]
        values = [hdf5_data[attr.value[i].item()].value[0][0]
                  for i in range(len(attr))] if len(attr) > 1 else [attr.value[0][0]]
        attrs[key] = values
    return attrs

# def collapse_col(row):
#     global resize_size
#     new_row = {}
#     new_row['img_name'] = list(row['img_name'])[0]
#     new_row['labels'] = row['label'].astype(np.str).str.cat(sep='_')
#     new_row['top'] = max(int(row['top'].min()),0)
#     new_row['left'] = max(int(row['left'].min()),0)
#     new_row['bottom'] = int(row['bottom'].max())
#     new_row['right'] = int(row['right'].max())
#     new_row['width'] = int(new_row['right'] - new_row['left'])
#     new_row['height'] = int(new_row['bottom'] - new_row['top'])
#     new_row['num_digits'] = len(row['label'].values)
#     return pd.Series(new_row,index=None)

# def image_data_constuctor(img_folder, img_bbox_data):
#     print('image data construction starting...')
#     imgs = []
#     for img_file in os.listdir(img_folder):
#         if img_file.endswith('.png'):
#             imgs.append([img_file,cv2.imread(os.path.join(img_folder,img_file))])
#     img_data = pd.DataFrame([],columns=['img_name','img_height','img_width','img','cut_img'])
#     print('finished loading images...starting image processing...')
#     for img_info in imgs:
#         row = img_bbox_data[img_bbox_data['img_name']==img_info[0]]
#         full_img = img_info[1] #cv2.normalize(cv2.cvtColor(cv2.resize(img_info[1],resize_size), cv2.COLOR_BGR2GRAY).astype(np.float64), 0, 1, cv2.NORM_MINMAX)
#         cut_img = full_img.copy()[int(row['top']):int(row['top']+row['height']),int(row['left']):int(row['left']+row['width']),...]
#         row_dict = {'img_name':[img_info[0]],'img_height':[img_info[1].shape[0]],'img_width':[img_info[1].shape[1]],'img':[full_img],'cut_img':[cut_img]}
#         img_data = pd.concat([img_data,pd.DataFrame.from_dict(row_dict,orient = 'columns')])
#     print('finished image processing...')
#     return img_data

# def get_name(index, hdf5_data):
#     name = hdf5_data['/digitStruct/name']
#     return ''.join([chr(v[0]) for v in hdf5_data[name[index][0]].value])

# def get_bbox(index, hdf5_data):
#     attrs = {}
#     item = hdf5_data['digitStruct']['bbox'][index].item()
#     for key in ['label', 'left', 'top', 'width', 'height']:
#         attr = hdf5_data[item][key]
#         values = [hdf5_data[attr.value[i].item()].value[0][0]
#                   for i in range(len(attr))] if len(attr) > 1 else [attr.value[0][0]]
#         attrs[key] = values
#     return attrs

# def img_boundingbox_data_constructor(mat_file):
#     f = h5py.File(mat_file,'r') 
#     all_rows = []
#     print('image bounding box data construction starting...')
#     bbox_df = pd.DataFrame([],columns=['height','img_name','label','left','top','width'])
#     for j in range(f['/digitStruct/bbox'].shape[0]):
#         img_name = get_name(j, f)
#         row_dict = get_bbox(j, f)
#         row_dict['img_name'] = img_name
#         all_rows.append(row_dict)
#         bbox_df = pd.concat([bbox_df,pd.DataFrame.from_dict(row_dict,orient = 'columns')])
#     bbox_df['bottom'] = bbox_df['top']+bbox_df['height']
#     bbox_df['right'] = bbox_df['left']+bbox_df['width']
#     print('finished image bounding box data construction...')
#     return bbox_df


# def construct_all_data(img_folder,mat_file_name,h5_name):
#     img_bbox_data = img_boundingbox_data_constructor(os.path.join(img_folder,mat_file_name))
#     img_bbox_data_grouped = img_bbox_data.groupby('img_name').apply(collapse_col) 
#     img_data = image_data_constuctor(img_folder, img_bbox_data_grouped)
#     print('done constructing main dataframes...starting grouping')
#     df1 = img_bbox_data_grouped.merge(img_data,on='img_name',how='left')
#     print('grouping done')
#     #df1.to_csv(os.path.join(img_folder,csv_name), index = False)
#     df1.to_hdf(os.path.join(img_folder,h5_name),'table')

# #construct_all_data(test_folder,'digitStruct.mat','test_data_processed.h5')
# construct_all_data(train_folder,'digitStruct.mat','train_data_processed.h5')
# #construct_all_data(extra_folder,'digitStruct.mat','extra_data_processed.h5') #takes a long time

# 製作label

In [2]:
import h5py  #匯入工具包
import numpy as np

In [8]:
# HDF5的讀取：
f = h5py.File('train/digitStruct.mat','r')
path = 'train_label'
data_num = len(f['digitStruct']['bbox'])


def mkdir(path):
    #判斷目錄是否存在
    #存在：True
    #不存在：False
    folder = os.path.exists(path)

    #判斷結果
    if not folder:
        #如果不存在，則建立新目錄
        os.makedirs(path)
        print('-----建立成功-----')

    else:
        #如果目錄已存在，則不建立，提示目錄已存在
        print(path+'目錄已存在')


mkdir(path)

-----建立成功-----


In [9]:
def makeLabel(file_name, h5, img_w, img_h, label_path):
    # for Yolo
    # label資料夾名稱要與data相同
    f = open(label_path.split('.')[0]+'.txt','w')
    idx = int(file_name)-1
    a = get_bbox(idx, h5)
    for i in range(len(a['label'])):
        label = int(a['label'][i])-1
        left = a['left'][i]
        top = a['top'][i]
        width = a['width'][i]
        height = a['height'][i]
        x = (left + width/2) / img_w
        y = (top + height/2) / img_h
        w = width / img_w
        h = height / img_h
        if x > 1:
            x=1
        if y > 1:
            y=1
        if w > 1:
            w=1
        if h>1:
            h=1
        f.write('{} {} {} {} {}'.format(label, x, y, w, h))
        f.write('\n')
        
    f.close()
        
    
    

In [10]:
originPath = 'train'
h5 = h5py.File('train/digitStruct.mat','r') 
# 列出指定路徑底下所有檔案(包含資料夾)
allFileList = os.listdir(originPath)

# 逐一查詢檔案清單
n = 0
for file in allFileList:
    file_name = file.split('.')[0]
    if os.path.isfile(os.path.join(originPath,file)):
        if file.endswith('png'):
            img = cv2.imread(os.path.join(originPath,file))
            img_h = img.shape[0]
            img_w = img.shape[1]
            n+=1
            label_path = 'train_label/'+file
            makeLabel(file_name, h5, img_w, img_h, label_path)
            
print(n)

33402


In [14]:
#HDF5的讀取：
f = h5py.File('train/digitStruct.mat','r') 
print(len(f['digitStruct']['bbox']))
a = get_name(12667, f)
b = get_bbox(12667,f)
print(a)
print(b)
print(len(b['label']))
print(b['label'][0])

33402
12668.png
{'label': [4.0, 10.0], 'left': [11.0, 17.0], 'top': [1.0, 1.0], 'width': [6.0, 8.0], 'height': [17.0, 11.0]}
2
4.0


In [15]:
label = []
for i in range(33402):
    a = get_name(i,f)
    b = get_bbox(i,f)
    for l in range(len(b['label'])):
        if b['label'][l] not in label:
            label.append(b['label'][l])
print(label)

In [10]:
allFileList2 = os.listdir('yolov3/data/images/train')
image_files = []
n = 0
for file in allFileList2:
        if file.endswith(".png"):
            image_files.append('data/images/train'+'/'+file)
            n+=1
            
            
f = open('train.txt', "w")
for image in image_files:
    f.write(image)
    f.write("\n")
f.close()

print(n)
#trian.txt最後一行要再手動編輯，把空白列刪除

33402


In [6]:
allFileList_t = os.listdir('test')
image_files_t = []
n = 0
for file in allFileList_t:
        image_files_t.append('data/images/test'+'/'+file)
        n+=1
            
            
f = open('test.txt', "w")
for image in image_files_t:
    f.write(image)
    f.write("\n")
f.close()

print(n)
#trian.txt最後一行要再手動編輯，把空白列刪除

13068


In [10]:
# 改檔名
test_path = 'yolov5/data/images/test/'

allFileList_test = os.listdir(test_path)

# 逐一查詢檔案清單
n = 0
for file in allFileList_test:
    file_name = file.split('.')[0]
    if file.endswith('png'):
        #print(file_name)
        #print(len(file_name))
        new_fname = file_name.zfill(5)
        os.rename(os.path.join(test_path, file), os.path.join(test_path, new_fname)+'.png')
        n+=1
            
print(n)

13068
